In [14]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib as plt

#volume needed
volume = list(range(0,100000, 1))

#This script tells you how many shares you need to purchase to dilute a loss
def dilute_losses(initial_volume, initial_price, current_price):
    ### Inputs and outputs
    #param initial_volume: the initial number of shares purchased
    #param initial_price: the inital price paid per share
    #param current_price: the current stock price
    #output: the number of shares needed to dilute the price and how much that would cost
    ###
    for i in volume:
        a = ((i * current_price) + (initial_volume * initial_price)) / (i + initial_volume)
        if a < (current_price + 0.01):
            print(i, 'new shares at a price of $', current_price, 'need to be purchased to mostly dilute your losses. This will cost roughly $',int(i * current_price))
            break
           

In [8]:
dilute_losses(100,0.3428,.2389)

940 new shares at a price of $ 0.2389 need to be purchased to mostly dilute your losses. This will cost roughly $ 224


In [3]:
#This script tells you the price per share to obtain a desired value
def new_price(initial_volume, initial_price, desired_value):
    ### Inputs and outputs
    #param initial_volume: the initial number of shares purchased
    #param initial_price: the inital price paid per share
    #param desired_value: the desired value of your holding
    #output: the stock price needed to achieve a given value and how much a stock needs to go up
    ###
    current_value = initial_volume * initial_price
    new_price = desired_value / initial_volume
    print('To obtain a desired value of $', desired_value, 'the price needs to go up to $', new_price, 'a change of $',
         new_price - initial_price)

In [4]:
new_price(100, 0.5, 100)
new_price(100, 0.5, 200)

To obtain a desired value of $ 100 the price needs to go up to $ 1.0 a change of $ 0.5
To obtain a desired value of $ 200 the price needs to go up to $ 2.0 a change of $ 1.5


In [5]:
#This script tells you how much of your profit needs to go to taxes, reinvestment, and savings
def distribution_of_profits(profit):
    print('$', profit * 0.4, 'needs to go to taxes')
    print('$', profit * 0.3, 'needs to go to reinvestment')
    print('$', profit * 0.3, 'needs to go to savings')

In [6]:
distribution_of_profits(100)

$ 40.0 needs to go to taxes
$ 30.0 needs to go to reinvestment
$ 30.0 needs to go to savings


In [61]:
#Connecting to yFinance and create a ticker dataframe


def create_ticker_df(ticker_symbols, start_date, end_date):
    ### Inputs and outputs
    #param ticker_symbols: a single or list of ticker symbols
    #param start_date: date in YYYY-M-D
    #param end_date: date in YYYY-M-D
    #output dataframe 
    ticker_df = []
    for i in ticker_symbols:
        temp_tick = yf.Ticker(i)
        temp_df = temp_tick.history(period='1d', start=start_date, end=end_date)
        temp_df['ticker'] = i
        ticker_df.append(temp_df)
    ticker_df = pd.concat(ticker_df, axis=0)
    return(ticker_df)

In [63]:
create_ticker_df(['MVIS','F'],'2020-4-1','2020-5-01')

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
Date,,,,,,,,
2020-04-01,0.17,0.57,0.17,0.20,50885400,0,0,MVIS
2020-04-02,0.22,0.33,0.20,0.22,18164100,0,0,MVIS
2020-04-03,0.23,0.24,0.19,0.21,3675000,0,0,MVIS
2020-04-06,0.21,0.27,0.19,0.22,5805200,0,0,MVIS
2020-04-07,0.23,0.23,0.21,0.23,2155200,0,0,MVIS
2020-04-08,0.20,0.23,0.20,0.23,1698500,0,0,MVIS
2020-04-09,0.23,0.23,0.21,0.21,3539000,0,0,MVIS
2020-04-13,0.21,0.22,0.20,0.22,3298400,0,0,MVIS
2020-04-14,0.21,0.34,0.21,0.32,16134800,0,0,MVIS
